In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('./melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

In [36]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Approach 1 (Drop Columns with Missing Values)

In [37]:
cols_with_missing_values = [col for col in X_train.columns if X_train[col].isnull().any()]

In [38]:
reduce_X_train  =X_train.drop(cols_with_missing_values, axis=1)
reduce_X_valid  =X_valid.drop(cols_with_missing_values, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduce_X_train, reduce_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


# Score from Approach 2 (Imputation)

In [39]:
from sklearn.impute import SimpleImputer
import pandas as pd
imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(imputer.fit_transform(X_valid))

In [40]:
imputed_X_train

,0,1,2,3,4,5,6,7,8,9,10,11
0,1.0,5.0,3182.0,1.0,1.0,1.0,0.0,153.764119,1940.000000,-37.85984,144.98670,13240.0
1,2.0,8.0,3016.0,2.0,2.0,1.0,193.0,153.764119,1964.839866,-37.85800,144.90050,6380.0
2,3.0,12.6,3020.0,3.0,1.0,1.0,555.0,153.764119,1964.839866,-37.79880,144.82200,3755.0
3,3.0,13.0,3046.0,3.0,1.0,1.0,265.0,153.764119,1995.000000,-37.70830,144.91580,8870.0
4,3.0,13.3,3020.0,3.0,1.0,2.0,673.0,673.000000,1970.000000,-37.76230,144.82720,4217.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10859,3.0,5.2,3056.0,3.0,1.0,2.0,212.0,153.764119,1964.839866,-37.77695,144.95785,11918.0
10860,3.0,10.5,3081.0,3.0,1.0,1.0,748.0,101.000000,1950.000000,-37.74160,145.04810,2947.0
10861,4.0,6.7,3058.0,4.0,2.0,2.0,441.0,255.000000,2002.000000,-37.73572,144.97256,11204.0
10862,3.0,12.0,3073.0,3.0,1.0,1.0,606.0,153.764119,1964.839866,-37.72057,145.02615,21650.0


In [41]:
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

In [42]:
print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
179816.89508731329


# core from Approach 3 (An Extension to Imputation)

In [43]:
X_train_copy = X_train.copy()
X_valid_copy = X_valid.copy()

for col in cols_with_missing_values:
    X_train_copy[col + '_was_missing'] = X_train_copy[col].isnull()
    X_valid_copy[col + '_was_missing'] = X_valid_copy[col].isnull()

In [44]:
myimputer = SimpleImputer()

imputed_x_train = pd.DataFrame(myimputer.fit_transform(X_train_copy))
imputed_x_valid = pd.DataFrame(myimputer.fit_transform(X_valid_copy))

In [45]:
imputed_x_train.columns = X_train_copy.columns
imputed_x_valid.columns = X_valid_copy.columns
imputed_x_valid.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car_was_missing,BuildingArea_was_missing,YearBuilt_was_missing
0,4.0,8.0,3016.0,4.0,2.0,2.0,450.0,190.000000,1910.000000,-37.86100,144.89850,6380.0,0.0,0.0,0.0
1,2.0,6.6,3011.0,2.0,1.0,0.0,172.0,81.000000,1900.000000,-37.81000,144.88960,2417.0,0.0,0.0,0.0
2,3.0,10.5,3020.0,3.0,1.0,1.0,581.0,144.756506,1964.064927,-37.76740,144.82421,4217.0,0.0,1.0,1.0
3,3.0,4.5,3181.0,2.0,2.0,1.0,128.0,134.000000,2000.000000,-37.85260,145.00710,7717.0,0.0,0.0,0.0
4,3.0,8.5,3044.0,3.0,2.0,2.0,480.0,144.756506,1964.064927,-37.72523,144.94567,7485.0,0.0,1.0,1.0


In [47]:
print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_x_train, imputed_x_valid, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
179986.2708570026
